In [ ]:
# Installation de Transformers et pyTorch
! pip install transformers datasets

In [ ]:
import torch
import transformers
torch.__version__, transformers.__version__

In [ ]:
torch.cuda.is_available()

## Constructions de modèles personnalisés

Vous pouvez modifier la configuration du modèle pour changer la façon dont un modèle est construit. La configuration spécifie les attributs d'un modèle, tels que le nombre de couches ou de têtes d'attention. Vous partez de zéro lorsque vous initialisez un modèle à partir d'une configuration personnalisée. Les attributs du modèle sont initialisés de manière aléatoire et vous devrez entraîner le modèle avant de pouvoir l'utiliser pour obtenir des résultats significatifs.

Commencez par importer `AutoConfig`, puis chargez le modèle pré-entraîné que vous voulez modifier. Dans `AutoConfig.from_pretrained()`, vous pouvez spécifier l'attribut que vous souhaitez modifier, tel que le nombre de têtes d'attention :

In [ ]:
from transformers import AutoConfig

my_config = AutoConfig.from_pretrained("distilbert-base-uncased", n_heads=12)

Créez un modèle personnalisé à partir de votre configuration avec `AutoModel.from_config()` :

In [ ]:
from transformers import AutoModel

my_model = AutoModel.from_config(my_config)

Consultez le guide [Créer une architecture personnalisée](https://huggingface.co/docs/transformers/main/fr/./create_a_model) pour plus d'informations sur la création de configurations personnalisées.

## Trainer - une boucle d'entraînement optimisée par PyTorch

Tous les modèles sont des [`torch.nn.Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module) standard, vous pouvez donc les utiliser dans n'importe quelle boucle d'entraînement typique. Bien que vous puissiez écrire votre propre boucle d'entraînement, 🤗 Transformers fournit une classe `Trainer` pour PyTorch, qui contient la boucle d'entraînement de base et ajoute des fonctionnalités supplémentaires comme l'entraînement distribué, la précision mixte, et plus encore.

En fonction de votre tâche, vous passerez généralement les paramètres suivants à `Trainer` :

1. Un `PreTrainedModel` ou un [`torch.nn.Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module):

   ```py
   >>> from transformers import AutoModelForSequenceClassification

   >>> model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")
   ```

2. `TrainingArguments` contient les hyperparamètres du modèle que vous pouvez changer comme le taux d'apprentissage, la taille due l'échantillon, et le nombre d'époques pour s'entraîner. Les valeurs par défaut sont utilisées si vous ne spécifiez pas d'hyperparamètres d'apprentissage :

   ```py
   >>> from transformers import TrainingArguments

   >>> training_args = TrainingArguments(
   ...     output_dir="path/to/save/folder/",
   ...     learning_rate=2e-5,
   ...     per_device_train_batch_size=8,
   ...     per_device_eval_batch_size=8,
   ...     num_train_epochs=2,
   ... )
   ```

3. Une classe de prétraitement comme un tokenizer, un processeur d'images ou un extracteur de caractéristiques :

   ```py
   >>> from transformers import AutoTokenizer

   >>> tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
   ```

4. Chargez un jeu de données :

   ```py
   >>> from datasets import load_dataset

   >>> dataset = load_dataset("rotten_tomatoes")  # doctest: +IGNORE_RESULT
   ```

5. Créez une fonction qui transforme le texte du jeu de données en token :

   ```py
   >>> def tokenize_dataset(dataset):
   ...     return tokenizer(dataset["text"])
   ```

   Puis appliquez-la à l'intégralité du jeu de données avec `map`:

   ```py
   >>> dataset = dataset.map(tokenize_dataset, batched=True)
   ```

6. Un `DataCollatorWithPadding` pour créer un échantillon d'exemples à partir de votre jeu de données :

   ```py
   >>> from transformers import DataCollatorWithPadding

   >>> data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
   ```

Maintenant, rassemblez tous ces éléments dans un `Trainer` :

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)  # doctest: +SKIP

Une fois que vous êtes prêt, appelez la fonction `train()` pour commencer l'entraînement :

In [ ]:
trainer.train()

<Tip>

Pour les tâches - comme la traduction ou la génération de résumé - qui utilisent un modèle séquence à séquence, utilisez plutôt les classes `Seq2SeqTrainer` et `Seq2SeqTrainingArguments`.

</Tip>

Vous pouvez personnaliser le comportement de la boucle d'apprentissage en redéfinissant les méthodes à l'intérieur de `Trainer`. Cela vous permet de personnaliser des caractéristiques telles que la fonction de perte, l'optimiseur et le planificateur. Consultez la documentation de `Trainer` pour savoir quelles méthodes peuvent être redéfinies. 

L'autre moyen de personnaliser la boucle d'apprentissage est d'utiliser les [Callbacks](https://huggingface.co/docs/transformers/main/fr/./main_classes/callbacks). Vous pouvez utiliser les callbacks pour intégrer d'autres bibliothèques et inspecter la boucle d'apprentissage afin de suivre la progression ou d'arrêter l'apprentissage plus tôt. Les callbacks ne modifient rien dans la boucle d'apprentissage elle-même. Pour personnaliser quelque chose comme la fonction de perte, vous devez redéfinir le `Trainer` à la place.